## Imports

In [4]:
from bs4 import BeautifulSoup
from time import sleep
from urllib.request import Request, urlopen

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from requests_html import HTMLSession

from tqdm import tqdm

import pandas as pd
import csv
import base64
from PIL import Image
from io import BytesIO
import os
import requests
import traceback
import re

## Selenium Setup

In [5]:
def openChrome():
    portz = 9995
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument(f'--remote-debugging-port={portz}')
    chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"])
    chromeOptions.add_argument("--disable-blink-features=AutomationControlled")
    chromeOptions.add_argument("--disable-blink-features")
    
    driver = webdriver.Chrome(executable_path=r"/Users/kylewang/Desktop/MLSA/chromedriver", options=chromeOptions)  
    driver.set_window_rect(0, 0, width=360, height=600)
    driver.set_page_load_timeout(66)
    return driver

## Scraper Code

In [6]:
df = pd.read_csv(r"/Users/kylewang/Desktop/MLSA/Scripts/google_images.csv", encoding= 'unicode_escape')
df = df.rename(columns={"ï»¿HTL": "HTL"})
df

,HTL,HOTEL_NAME,ADDRESS,Google_Search,Google_Image_Href,Google_Streetview_Href
0,DA75201400.OLIVE.ST.................,SHERATON DALLAS,400 OLIVE ST,SHERATON DALLAS 400 OLIVE ST,NaN,NaN
1,GR760511501.GAYLORD.TRL.............,GAYLORD TEXAS RESORT & CONF CTR,1501 GAYLORD TRL,GAYLORD TEXAS RESORT & CONF CTR 1501 GAYLORD TRL,NaN,NaN
2,DA752072201.N.STEMMONS.FWY..........,HILTON ANATOLE HOTEL,2201 N STEMMONS FWY,HILTON ANATOLE HOTEL 2201 N STEMMONS FWY,NaN,NaN
3,HO770101600.LAMAR.ST................,HILTON AMERICAS CONVENTION CTR,1600 LAMAR ST,HILTON AMERICAS CONVENTION CTR 1600 LAMAR ST,NaN,NaN
4,DA75207300.E.REUNION.BLVD...........,HYATT REGENCY DALLAS DOWNTOWN,300 E REUNION BLVD,HYATT REGENCY DALLAS DOWNTOWN 300 E REUNION BLVD,NaN,NaN
...,...,...,...,...,...,...
7526,HO770707700.WILLOW.CHASE.BLVD.APT.17,MARRIOTT EXECUSTAY,7700 WILLOW CHASE BLVD APT 1733,MARRIOTT EXECUSTAY 7700 WILLOW CHASE BLVD APT ...,NaN,NaN
7527,DR786202390.PROCHNOW.RD.............,MT GAINOR INN,2390 PROCHNOW RD,MT GAINOR INN 2390 PROCHNOW RD,NaN,NaN
7528,GO78629222.SAINT.PETER.ST...........,BELLE OAKS INN,222 SAINT PETER ST,BELLE OAKS INN 222 SAINT PETER ST,NaN,NaN
7529,HO770247855.KATY.FWY................,CLSD WELLESLEY INN & SUITES,7855 KATY FWY,CLSD WELLESLEY INN & SUITES 7855 KATY FWY,NaN,NaN


In [7]:
df = df[7435:]
df

In [9]:
def save_img(url, output_path):
    try:
        if "data:image/" in url:
            url = url.replace('data:image/jpeg;base64,',"").replace('data:image/jpg;base64,',"")
            image_data = base64.b64decode(url)
            image = Image.open(BytesIO(image_data))
            image.save(output_path, format='JPEG')
        else:
            response = requests.get(url)
            response.raise_for_status()
            with open(output_path, 'wb') as file:
                file.write(response.content)
    except:
        print(traceback.format_exc())
        print("Fail Down IMG")

In [6]:
f_img_1 = "htl_google"
f_img_2 = "htl_streetview"
try:
    os.makedirs(f_img_1)
    print(f"Folder created successfully.")
except FileExistsError:
    print(f"Folder already exists.")
try:
    os.makedirs(f_img_2)
    print(f"Folder created successfully.")
except FileExistsError:
    print(f"Folder already exists.")

driver = openChrome()
input()

with tqdm(total=len(df)) as pbar:
    for index, row in df.iterrows():
        sech = row['Google_Search']
        htl = row['HTL']
        for tz in range(9):
            try:
                driver.get(f"https://www.google.com/search?q={sech}")
                sleep(1)
                element = WebDriverWait(driver,33).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='logo']"))
                )
                sleep(1)

                page_source  = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')

                pattern = re.compile(r'photos$')
                img1 = soup.find("img", attrs={"alt": pattern})
                if img1 != None:
                    img1 = img1.get("src")
                    name_img1 = htl + "_google"
                    print(name_img1)
                    save_img(img1, f"{f_img_1}/{name_img1}.png")

                pattern = re.compile(r'outside$')
                img2 = soup.find("img", attrs={"alt":pattern})
                if img2 != None:
                    img2 = img2.get("src")
                    name_img2 = htl + "_streetview"
                    print(name_img2)
                    save_img(img2, f"{f_img_2}/{name_img2}.png")
                break
            except:# ZeroDivisionError  as e:
                print(traceback.format_exc())
                print("Fail Try Again")
                sleep(3)
        pbar.update(1)

Folder already exists.
Folder already exists.


C:\Users\kylew\AppData\Local\Temp\ipykernel_45100\3098742563.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\kylew\OneDrive\Desktop\Advent\chromedriver.exe", options=chromeOptions)


  1%|          | 1/96 [00:04<06:24,  4.05s/it]

PO78598SOUTH.SHORE.DRIVE............_google
PO78598SOUTH.SHORE.DRIVE............_streetview


  2%|▏         | 2/96 [00:08<06:15,  3.99s/it]

...

DR786202390.PROCHNOW.RD............._google


 98%|█████████▊| 94/96 [05:20<00:06,  3.29s/it]

GO78629222.SAINT.PETER.ST..........._google
GO78629222.SAINT.PETER.ST..........._streetview


100%|██████████| 96/96 [05:27<00:00,  3.41s/it]

SO78078SOUTH.PADRE.ISLAND..........._google
SO78078SOUTH.PADRE.ISLAND..........._streetview
